In [23]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
import plotly.express as px
import plotly
import plotly.graph_objs as go


import os
if not os.path.exists("images"):
    os.mkdir("images")

## Read Datasets

In [24]:
#read super Covid-19 dataframe. 
USCovid = pd.read_csv("../../data/output/covid.csv")
#read country cases dataset and choose countries with similar pop as US
CountryNewCases = pd.read_csv("https://covid.ourworldindata.org/data/ecdc/new_cases.csv")
CountryNewDeaths = pd.read_csv("https://covid.ourworldindata.org/data/ecdc/new_deaths.csv")
CountryPop = pd.read_csv("https://covid.ourworldindata.org/data/ecdc/locations.csv")

CountryNewCases = CountryNewCases[["date","Bangladesh","Indonesia","Pakistan","Brazil","Nigeria"]]
CountryNewDeaths = CountryNewDeaths[["date","Bangladesh","Indonesia","Pakistan","Brazil","Nigeria"]]
PopArray = ["Bangladesh","Indonesia","Pakistan","Brazil","Nigeria"]
CountryPop = CountryPop.loc[CountryPop["location"].isin(PopArray)]

MyFivePop = CountryPop.drop(["countriesAndTerritories","continent","population_year"],axis = 1) 

#drop unneccesary variables in Super covid-19 dataframe. 
USCovid = USCovid.drop(["Unnamed: 0","countyFIPS","stateFIPS"], axis=1)

## Sum up United States Cases together and get New cases and New deaths Normalized by cases per 1 million people. 

In [25]:
#Removes "Statewide unallocated" value in covid dataframe
covid_df = USCovid.drop(USCovid.loc[USCovid['County Name'] == "Statewide Unallocated"].index)

#sums up every covid cases from every county and state to one row. 
sums = covid_df.select_dtypes(pd.np.number).sum(axis = 0).rename('UnitedStates')
sums = sums.to_frame()



<ipython-input-25-2916e9cd5f11>:5: FutureWarning:

The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead



In [26]:
#get Population value of US and all countries to normalize in later cells. 
USPopulation = sums.values[0]
Bang_Pop = MyFivePop.values[0,1]
Brazil_Pop = MyFivePop.values[1,1]
Indon_Pop = MyFivePop.values[2,1]
Nigeria_Pop = MyFivePop.values[3,1]
Pak_Pop = MyFivePop.values[4,1]



In [27]:
#seperate cases and deaths. 
US_cases = sums.filter(regex = "x", axis = 0)
US_deaths = sums.filter(regex = "y",axis = 0)

In [28]:
#calculate only new cases 
NewUScases = US_cases.diff()
NewUSdeaths = US_deaths.diff()

print(NewUScases[260:])
print(NewUSdeaths[260:])

            UnitedStates
10/8/20_x        56452.0
10/9/20_x        56721.0
10/10/20_x       54093.0
10/11/20_x       43191.0
10/12/20_x       42553.0
10/13/20_x       50383.0
10/14/20_x       59676.0
10/15/20_x       66676.0
10/16/20_x       63730.0
10/17/20_x       59993.0
10/18/20_x       46978.0
10/19/20_x       56778.0
10/20/20_x       60112.0
10/21/20_x       62516.0
10/22/20_x       77365.0
10/23/20_x       82828.0
10/24/20_x       70945.0
10/25/20_x       62897.0
10/26/20_x       68068.0
10/27/20_x       71009.0
10/28/20_x       78233.0
10/29/20_x       88819.0
10/30/20_x       96263.0
10/31/20_x       73389.0
11/1/20_x        82789.0
11/2/20_x        85621.0
11/3/20_x        91790.0
11/4/20_x       107428.0
11/5/20_x       117655.0
11/6/20_x       132276.0
11/7/20_x       112610.0
11/8/20_x       103701.0
11/9/20_x       127172.0
            UnitedStates
10/8/20_y          928.0
10/9/20_y          899.0
10/10/20_y         594.0
10/11/20_y         500.0
10/12/20_y         283.0


In [29]:
#calculate and Normalize dataset to cases per 50 million people. 
norm_val = 50_000_000
USNormalize = lambda x: (x / USPopulation) * norm_val
Norm_NewUScases = NewUScases.transform(USNormalize).round(0)
Norm_NewUSdeaths = NewUSdeaths.transform(USNormalize).round(0)
print(Norm_NewUScases)
print(Norm_NewUSdeaths)

           UnitedStates
1/22/20_x           NaN
1/23/20_x           0.0
1/24/20_x           0.0
1/25/20_x           0.0
1/26/20_x           0.0
...                 ...
11/5/20_x       17922.0
11/6/20_x       20149.0
11/7/20_x       17154.0
11/8/20_x       15797.0
11/9/20_x       19372.0

[293 rows x 1 columns]
           UnitedStates
1/22/20_y           NaN
1/23/20_y           0.0
1/24/20_y           0.0
1/25/20_y           0.0
1/26/20_y           0.0
...                 ...
11/5/20_y         171.0
11/6/20_y         167.0
11/7/20_y         138.0
11/8/20_y          67.0
11/9/20_y         139.0

[293 rows x 1 columns]


In [30]:
#changed date format for cases
Norm_NewUScases = Norm_NewUScases.T
Norm_NewUScases=Norm_NewUScases.rename(columns=lambda x: (pd.to_datetime(x.split('_')[0], errors= 'ignore')), errors='raise')
Norm_NewUScases = Norm_NewUScases.T
print(Norm_NewUScases.head())

#changed date format for deaths
Norm_NewUSdeaths = Norm_NewUSdeaths.T
Norm_NewUSdeaths=Norm_NewUSdeaths.rename(columns=lambda x: (pd.to_datetime(x.split('_')[0], errors= 'ignore')), errors='raise')
Norm_NewUSdeaths = Norm_NewUSdeaths.T
print(Norm_NewUSdeaths.head())

            UnitedStates
2020-01-22           NaN
2020-01-23           0.0
2020-01-24           0.0
2020-01-25           0.0
2020-01-26           0.0
            UnitedStates
2020-01-22           NaN
2020-01-23           0.0
2020-01-24           0.0
2020-01-25           0.0
2020-01-26           0.0


## Get weekly average of cases

In [31]:
WeeklyUS_cases = Norm_NewUScases.resample('W').mean().round(0).astype(int).rename_axis('Week')
WeeklyUS_deaths = Norm_NewUSdeaths.resample('W').mean().round(0).astype(int).rename_axis('Week')

In [32]:
print(WeeklyUS_cases)
print(WeeklyUS_deaths)

            UnitedStates
Week                    
2020-01-26             0
2020-02-02             0
2020-02-09             0
2020-02-16             0
2020-02-23             0
2020-03-01             0
2020-03-08            10
2020-03-15            65
2020-03-22           643
2020-03-29          2257
2020-04-05          4170
2020-04-12          4767
2020-04-19          4299
2020-04-26          4521
2020-05-03          4167
2020-05-10          3668
2020-05-17          3365
2020-05-24          3380
2020-05-31          3155
2020-06-07          3283
2020-06-14          3262
2020-06-21          4003
2020-06-28          5751
2020-07-05          7173
2020-07-12          8979
2020-07-19         10008
2020-07-26          9824
2020-08-02          9254
2020-08-09          8105
2020-08-16          7594
2020-08-23          6459
2020-08-30          6273
2020-09-06          6213
2020-09-13          5224
2020-09-20          6205
2020-09-27          6664
2020-10-04          6472
2020-10-11          7362


## Find Mean, Median and Mode for all weeks of US. 

In [33]:
#Calculates mean median and mode for cases and deaths in US. 
US_MeanCases = WeeklyUS_cases.mean().round(0)
US_MedianCases = WeeklyUS_cases.median()
US_ModeCases = WeeklyUS_cases.mode()


US_MeanDeaths = WeeklyUS_deaths.mean().round(0)
US_MedianDeaths = WeeklyUS_deaths.median()
US_ModeDeaths = WeeklyUS_deaths.mode()

US_ModeCases = US_ModeCases.values.tolist()
US_ModeDeaths = US_ModeDeaths.values.tolist()


## Calculate Country Mean, Median, and Mode weekly statistics to compare with U.S

In [44]:
#fill NaN with zero for country dataset.
Country_NewCases = CountryNewCases[22:290].fillna(0)
Country_NewDeaths = CountryNewDeaths[22:290].fillna(0)

Country_NewCases.to_csv('../../data/output/Country_NewCases.csv',index=False)
Country_NewDeaths.to_csv('../../data/output/Country_NewDeaths.csv',index=False)
print(Country_NewCases)
print(Country_NewDeaths)

           date  Bangladesh  Indonesia  Pakistan  Brazil  Nigeria
22   2020-01-22         0.0        0.0       0.0       0      0.0
23   2020-01-23         0.0        0.0       0.0       0      0.0
24   2020-01-24         0.0        0.0       0.0       0      0.0
25   2020-01-25         0.0        0.0       0.0       0      0.0
26   2020-01-26         0.0        0.0       0.0       0      0.0
..          ...         ...        ...       ...     ...      ...
285  2020-10-11      1203.0     4294.0     666.0   26749    111.0
286  2020-10-12      1193.0     4497.0       0.0   12345    163.0
287  2020-10-13      1472.0     3267.0     916.0    8426    164.0
288  2020-10-14      1537.0     3906.0     615.0   10220    225.0
289  2020-10-15      1684.0     4127.0     755.0   27235    179.0

[268 rows x 6 columns]
           date  Bangladesh  Indonesia  Pakistan  Brazil  Nigeria
22   2020-01-22         0.0        0.0       0.0       0      0.0
23   2020-01-23         0.0        0.0       0.0    

## Normalize Cases and Death by 1 million population. 

In [35]:
#calculate and Normalize dataset to cases per 1 million people. 
Norm_CountryCases = Country_NewCases.assign(Bangladesh = lambda x: (x['Bangladesh'] / Bang_Pop) * norm_val).round(0)
Norm_CountryCases = Norm_CountryCases.assign(Indonesia = lambda x: (x['Indonesia'] / Indon_Pop) * norm_val).round(0)
Norm_CountryCases = Norm_CountryCases.assign(Pakistan = lambda x: (x['Pakistan'] / Pak_Pop) * norm_val).round(0)
Norm_CountryCases = Norm_CountryCases.assign(Brazil = lambda x: (x['Brazil'] / Brazil_Pop ) * norm_val).round(0)
Norm_CountryCases = Norm_CountryCases.assign(Nigeria = lambda x: (x['Nigeria'] / Nigeria_Pop ) * norm_val).round(0)

Norm_Country_NewDeaths = Country_NewDeaths.assign(Bangladesh = lambda x: (x['Bangladesh'] / Bang_Pop) * norm_val).round(0)
Norm_Country_NewDeaths = Norm_Country_NewDeaths.assign(Indonesia = lambda x: (x['Indonesia'] / Indon_Pop) * norm_val).round(0)
Norm_Country_NewDeaths = Norm_Country_NewDeaths.assign(Pakistan = lambda x: (x['Pakistan'] / Pak_Pop) * norm_val).round(0)
Norm_Country_NewDeaths = Norm_Country_NewDeaths.assign(Brazil = lambda x: (x['Brazil'] / Brazil_Pop ) * norm_val).round(0)
Norm_Country_NewDeaths = Norm_Country_NewDeaths.assign(Nigeria = lambda x: (x['Nigeria'] / Nigeria_Pop ) * norm_val).round(0)



print(Norm_CountryCases)
print(Norm_Country_NewDeaths)


           date  Bangladesh  Indonesia  Pakistan  Brazil  Nigeria
22   2020-01-22         0.0        0.0       0.0     0.0      0.0
23   2020-01-23         0.0        0.0       0.0     0.0      0.0
24   2020-01-24         0.0        0.0       0.0     0.0      0.0
25   2020-01-25         0.0        0.0       0.0     0.0      0.0
26   2020-01-26         0.0        0.0       0.0     0.0      0.0
..          ...         ...        ...       ...     ...      ...
285  2020-10-11       365.0      785.0     151.0  6292.0     27.0
286  2020-10-12       362.0      822.0       0.0  2904.0     40.0
287  2020-10-13       447.0      597.0     207.0  1982.0     40.0
288  2020-10-14       467.0      714.0     139.0  2404.0     55.0
289  2020-10-15       511.0      754.0     171.0  6406.0     43.0

[268 rows x 6 columns]
           date  Bangladesh  Indonesia  Pakistan  Brazil  Nigeria
22   2020-01-22         0.0        0.0       0.0     0.0      0.0
23   2020-01-23         0.0        0.0       0.0    

In [36]:
#set index as a datetime to easily calculate by weekly basis. 
Norm_CountryCases['date'] = pd.to_datetime(Norm_CountryCases['date'])
Norm_CountryCases = Norm_CountryCases.set_index('date')

Norm_Country_NewDeaths['date'] = pd.to_datetime(Norm_Country_NewDeaths['date'])
Norm_Country_NewDeaths = Norm_Country_NewDeaths.set_index('date')

## Find average of Cases and deaths for each week. 

In [37]:
#calculates weekly average of cases and deaths

WeeklyCountries_cases = Norm_CountryCases.resample('W').mean().round(0).astype(int).rename_axis('Week')
WeeklyCountries_deaths = Norm_Country_NewDeaths.resample('W').mean().round(0).astype(int).rename_axis('Week')

print(WeeklyCountries_cases)
print(WeeklyCountries_deaths)

            Bangladesh  Indonesia  Pakistan  Brazil  Nigeria
Week                                                        
2020-01-26           0          0         0       0        0
2020-02-02           0          0         0       0        0
2020-02-09           0          0         0       0        0
2020-02-16           0          0         0       0        0
2020-02-23           0          0         0       0        0
2020-03-01           0          0         0       0        0
2020-03-08           0          0         0       0        0
2020-03-15           0          2         1       4        0
2020-03-22           1          9        20      34        1
2020-03-29           1         19        25      93        3
2020-04-05           1         25        34     214        4
2020-04-12          18         46        84     351        4
2020-04-19          72         63        96     533        8
2020-04-26         124         62       153     736       22
2020-05-03         165  

## Calculate Mean, Median, and Mode of cases and deaths for all 5 countries. 

In [38]:
FiveCountry_MeanCases = WeeklyCountries_cases.mean().round(0)
FiveCountry_MedianCases = WeeklyCountries_cases.median()
FiveCountry_ModeCases = WeeklyCountries_cases.mode()

FiveCountry_MeanDeaths = WeeklyCountries_deaths.mean().round(0)
FiveCountry_MedianDeaths = WeeklyCountries_deaths.median()
FiveCountry_ModeDeaths = WeeklyCountries_deaths.mode()

FiveCountry_ModeCases = FiveCountry_ModeCases.values.tolist()
FiveCountry_ModeDeaths = FiveCountry_ModeDeaths.values.tolist()


## Puts Mean,Median and Mode values in a list to put into a dataframe for easier comparisions. 

In [39]:
ListCase = [["United States",US_MeanCases[0],US_MedianCases[0],US_ModeCases[0][0]],
        ["Bangladesh", FiveCountry_MeanCases[0],FiveCountry_MedianCases[0],FiveCountry_ModeCases[0][0]],
        ["Indonesia", FiveCountry_MeanCases[1],FiveCountry_MedianCases[1],FiveCountry_ModeCases[0][1]],
        ["Pakistan", FiveCountry_MeanCases[2],FiveCountry_MedianCases[2],FiveCountry_ModeCases[0][2]],
        ["Brazil", FiveCountry_MeanCases[3],FiveCountry_MedianCases[3],FiveCountry_ModeCases[0][3]],
       ["Nigeria", FiveCountry_MeanCases[4],FiveCountry_MedianCases[4],FiveCountry_ModeCases[0][4]]]


ListDeaths = [["United States",US_MeanDeaths[0],US_MedianDeaths[0],US_ModeDeaths[0][0]],
        ["Bangladesh", FiveCountry_MeanDeaths[0],FiveCountry_MedianDeaths[0],FiveCountry_ModeDeaths[0][0]],
        ["Indonesia", FiveCountry_MeanDeaths[1],FiveCountry_MedianDeaths[1],FiveCountry_ModeDeaths[0][1]],
        ["Pakistan", FiveCountry_MeanDeaths[2],FiveCountry_MedianDeaths[2],FiveCountry_ModeDeaths[0][2]],
        ["Brazil", FiveCountry_MeanDeaths[3],FiveCountry_MedianDeaths[3],FiveCountry_ModeDeaths[0][3]],
       ["Nigeria", FiveCountry_MeanDeaths[4],FiveCountry_MedianDeaths[4],FiveCountry_ModeDeaths[0][4]]]

## WeeklyStatistics Comparision of Cases per 50 mil people.

In [40]:
CountryWeeklyStatisticsCase = pd.DataFrame(ListCase, columns = ['Country','Mean', 'Median', 'Mode'])

CountryWeeklyStatisticsCase


,Country,Mean,Median,Mode
0,United States,5424.0,4767.0,0
1,Bangladesh,431.0,447.0,0
2,Indonesia,239.0,124.0,0
3,Pakistan,268.0,140.0,0
4,Brazil,4467.0,5076.0,0
5,Nigeria,55.0,42.0,0


## WeeklyStatistics Comparision of Deaths per 50 mil people. 

In [41]:
CountryWeeklyStatisticsDeath = pd.DataFrame(ListDeaths, columns = ['Country','Mean', 'Median', 'Mode'])

CountryWeeklyStatisticsDeath

,Country,Mean,Median,Mode
0,United States,122.0,120.0,0
1,Bangladesh,6.0,7.0,0
2,Indonesia,8.0,7.0,0
3,Pakistan,5.0,2.0,0
4,Brazil,132.0,164.0,0
5,Nigeria,1.0,1.0,0


# Analysis of Weekly Statistics:
## For United States, For both New Cases and New Death, the Mean is greater than the Median, So we can assume that its distrubution will be Right-Skewed. 

### - For Number of New Cases, Every country except for Brazil and Bangladesh seems to have a right-skewed distrubition.
### - For Number of New Deaths, United States, Indonesia and pakistan is assumed to have a  right-skewed distrubtion while Brazil and Bangladesh is assumed to have a left-skewed distrubition.
### - For Number of New Deaths, Nigeria seems to have a normal distrubtion. 

## United States has the highest average amount of new cases for every 50 million people, compared to other countries with similar populations, with Brazil not that far behind. 

## Compared to other countries, United States and Brazil's amount of new cases and deaths are really high. 

## Brazil has the highest average of new deaths on a weekly basis with United States closely behind. 

# Plot Daily Trends

In [42]:

trace0 = go.Scatter(x=Norm_NewUScases.index, y=Norm_NewUScases["UnitedStates"], 
                    name="US", 
                    mode="lines+markers") 

trace1 = go.Scatter(x=Norm_CountryCases.index, y=Norm_CountryCases["Bangladesh"], 
                    name="Bangladesh", 
                    mode="lines+markers")  

trace2 = go.Scatter(x=Norm_CountryCases.index, y=Norm_CountryCases["Indonesia"], 
                    name="Indonesia", 
                    mode="lines+markers") 

trace3 = go.Scatter(x=Norm_CountryCases.index, y=Norm_CountryCases["Pakistan"], 
                    name="Pakistan", 
                    mode="lines+markers") 

trace4 = go.Scatter(x=Norm_CountryCases.index, y=Norm_CountryCases["Brazil"], 
                    name="Brazil", 
                    mode="lines+markers") 

trace5 = go.Scatter(x=Norm_CountryCases.index, y=Norm_CountryCases["Nigeria"], 
                    name="Nigeria", 
                    mode="lines+markers")

mydata = go.Data([trace0, trace1,trace2, trace3,trace4,trace5])

mylayout = go.Layout(
    title="Daily Trend of Cases per 50 million population"
)

fig = go.Figure(data=mydata, layout=mylayout)

plotly.offline.iplot(fig, filename = '')

fig.write_image("images/DailyTrend_NewCases_Countries.png")

C:\Users\CodingTheories\Anaconda3\envs\CSC_405\lib\site-packages\plotly\graph_objs\_deprecations.py:31: DeprecationWarning:

plotly.graph_objs.Data is deprecated.
Please replace it with a list or tuple of instances of the following types
  - plotly.graph_objs.Scatter
  - plotly.graph_objs.Bar
  - plotly.graph_objs.Area
  - plotly.graph_objs.Histogram
  - etc.




ValueError: 
The orca executable is required to export figures as static images,
but it could not be found on the system path.

Searched for executable 'orca' on the following path:
    C:\Users\CodingTheories\Anaconda3\envs\CSC_405
    C:\Users\CodingTheories\Anaconda3\envs\CSC_405\Library\mingw-w64\bin
    C:\Users\CodingTheories\Anaconda3\envs\CSC_405\Library\usr\bin
    C:\Users\CodingTheories\Anaconda3\envs\CSC_405\Library\bin
    C:\Users\CodingTheories\Anaconda3\envs\CSC_405\Scripts
    C:\Users\CodingTheories\Anaconda3\envs\CSC_405\bin
    C:\Users\CodingTheories\Anaconda3\condabin
    C:\Users\CodingTheories\Anaconda3
    C:\Users\CodingTheories\Anaconda3\Library\mingw-w64\bin
    C:\Users\CodingTheories\Anaconda3\Library\usr\bin
    C:\Users\CodingTheories\Anaconda3\Library\bin
    C:\Users\CodingTheories\Anaconda3\Scripts
    C:\WINDOWS\system32
    C:\WINDOWS
    C:\WINDOWS\System32\Wbem
    C:\WINDOWS\System32\WindowsPowerShell\v1.0
    C:\WINDOWS\System32\OpenSSH
    C:\P

In [ ]:

trace0 = go.Scatter(x=WeeklyUS_cases.index, y=WeeklyUS_cases["UnitedStates"], 
                    name="US", 
                    mode="lines+markers") 

trace1 = go.Scatter(x=WeeklyCountries_cases.index, y=WeeklyCountries_cases["Bangladesh"], 
                    name="Bangladesh", 
                    mode="lines+markers")  

trace2 = go.Scatter(x=WeeklyCountries_cases.index, y=WeeklyCountries_cases["Indonesia"], 
                    name="Indonesia", 
                    mode="lines+markers") 

trace3 = go.Scatter(x=WeeklyCountries_cases.index, y=WeeklyCountries_cases["Pakistan"], 
                    name="Pakistan", 
                    mode="lines+markers") 

trace4 = go.Scatter(x=WeeklyCountries_cases.index, y=WeeklyCountries_cases["Brazil"], 
                    name="Brazil", 
                    mode="lines+markers") 

trace5 = go.Scatter(x=WeeklyCountries_cases.index, y=WeeklyCountries_cases["Nigeria"], 
                    name="Nigeria", 
                    mode="lines+markers")

mydata = go.Data([trace0, trace1,trace2, trace3,trace4,trace5])

mylayout = go.Layout(
    title="Weekly Trend of Cases per 50 million population"
)

fig2 = go.Figure(data=mydata, layout=mylayout)

plotly.offline.iplot(fig2, filename = '')

fig2.write_image("images/WeeklyTrend_NewCases_Countries,.png")

In [ ]:
trace0 = go.Scatter(x=Norm_NewUSdeaths.index, y=Norm_NewUSdeaths["UnitedStates"], 
                    name="US", 
                    mode="lines+markers") 

trace1 = go.Scatter(x=Norm_Country_NewDeaths.index, y=Norm_Country_NewDeaths["Bangladesh"], 
                    name="Bangladesh", 
                    mode="lines+markers")  

trace2 = go.Scatter(x=Norm_Country_NewDeaths.index, y=Norm_Country_NewDeaths["Indonesia"], 
                    name="Indonesia", 
                    mode="lines+markers") 

trace3 = go.Scatter(x=Norm_Country_NewDeaths.index, y=Norm_Country_NewDeaths["Pakistan"], 
                    name="Pakistan", 
                    mode="lines+markers") 

trace4 = go.Scatter(x=Norm_Country_NewDeaths.index, y=Norm_Country_NewDeaths["Brazil"], 
                    name="Brazil", 
                    mode="lines+markers") 

trace5 = go.Scatter(x=Norm_Country_NewDeaths.index, y=Norm_Country_NewDeaths["Nigeria"], 
                    name="Nigeria", 
                    mode="lines+markers")

mydata = go.Data([trace0, trace1,trace2, trace3,trace4,trace5])

mylayout = go.Layout(
    title="Daily Trend of Deaths per 50 million population"
)

fig3 = go.Figure(data=mydata, layout=mylayout)

plotly.offline.iplot(fig3, filename = '')

fig3.write_image("images/DailyTrend_NewDeaths_Countries,.png")

In [ ]:

trace0 = go.Scatter(x=WeeklyUS_deaths.index, y=WeeklyUS_deaths["UnitedStates"], 
                    name="US", 
                    mode="lines+markers") 

trace1 = go.Scatter(x=WeeklyCountries_deaths.index, y=WeeklyCountries_deaths["Bangladesh"], 
                    name="Bangladesh", 
                    mode="lines+markers")  

trace2 = go.Scatter(x=WeeklyCountries_deaths.index, y=WeeklyCountries_deaths["Indonesia"], 
                    name="Indonesia", 
                    mode="lines+markers") 

trace3 = go.Scatter(x=WeeklyCountries_deaths.index, y=WeeklyCountries_deaths["Pakistan"], 
                    name="Pakistan", 
                    mode="lines+markers") 

trace4 = go.Scatter(x=WeeklyCountries_deaths.index, y=WeeklyCountries_deaths["Brazil"], 
                    name="Brazil", 
                    mode="lines+markers") 

trace5 = go.Scatter(x=WeeklyCountries_deaths.index, y=WeeklyCountries_deaths["Nigeria"], 
                    name="Nigeria", 
                    mode="lines+markers")

mydata = go.Data([trace0, trace1,trace2, trace3,trace4,trace5])

mylayout = go.Layout(
    title="Weekly Trend of Deaths per 50 million population"
)

fig4 = go.Figure(data=mydata, layout=mylayout)

plotly.offline.iplot(fig4, filename = '')

fig4.write_image("images/WeeklyTrend_NewDeaths_Countries,.png")

# Peak Week Of Cases:
### - United States: Week of July 12 - July 19, 2020 
### - Bangladesh:  Week of June 28 - July 5, 2020
### - Indonesia:  Week of September 20 - September 27,  2020
### - Pakistan:  Week of June 7 - June 14, 2020
### - Brazil:  Week of July 19 - July 26, 2020
### - Nigeria:  Week of June 21 - June 28, 2020


# Peak Week Of Deaths:
### - United States: Week of April 12 - April 19 2020 
### - Bangladesh: Week of August 23 - August 30, 2020  
### - Indonesia: Week of September 20 - September 27, 2020 
### - Pakistan: Week of June 14 - June 21, 2020 
### - Brazil: Week of July 19 - July 26, 2020 
### - Nigeria: Week of July 19 - July 26, 2020 